In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tqdm.notebook as tq
from time import time
from time import sleep
import sys

In [2]:
#read in the data
path = r"C:\Users\acros\.spyder-py3\ml-20m\ml-20m\ratings.csv"
rating_data = pd.read_csv(path, nrows=2000000)
rating_data = rating_data.rename(columns={'userId':'user_id', 'movieId': 'item_id'})
rating_data = rating_data.groupby('user_id').filter(lambda x: len(x) >= 5)
rating_data = rating_data.groupby('item_id').filter(lambda x: len(x) >= 5)
num_users = len(rating_data.user_id.unique())
num_items = len(rating_data.item_id.unique())
sparsity = len(rating_data) / (num_users * num_items)
print("Num Users: {}, Num Items: {}, Sparsity: {:.3f}".format(num_users, num_items, sparsity))

Num Users: 6743, Num Items: 8613, Sparsity: 0.017


In [3]:
#create training dataset
user_list = rating_data.user_id.unique()
item_list = rating_data.item_id.unique()
train_dict = {}
test_dict = {}
used_items_dict = {}
for user in tq.tqdm(user_list):
    user_items = rating_data[rating_data.user_id==user].item_id    #get the items for a particular user
    
    #make a random sample of positive and negative items
    train_items = user_items.sample(frac=0.50)
    test_items = list(set(user_items).difference(train_items))
    negative_items = np.random.choice(list(set(item_list).difference(user_items)), size=len(train_items))
    
    #create training and testing data
    train_items = list(zip(train_items, negative_items))
    train_dict[user] = train_items
    test_dict[user] = test_items
    used_items_dict[user] = [train_items, negative_items, test_dict]    #for evaluation later

In [4]:
del(rating_data)    #no longer needed as we have our dictionaries

In [217]:
def sigmoid(x):
    prob = 1 / (1 + np.exp(-x))
    return(prob)

class BPR:
    def __init__(self, learning_rate=0.05, maxiter=10, n_factors=15, lam=0.05):
        self.learning_rate = learning_rate
        self.maxiter=maxiter
        self.n_factors=n_factors
        self.lam = lam
        
    def fit(self, train_data, items):
        #initialize
        self.train_data = train_data
        self.items = items
        self.users = list(train_data.keys())
        self.U = {u : np.random.randn(1, self.n_factors)/1e6 for u in self.users}
        self.V = {j : np.random.randn(1, self.n_factors)/1e6 for j in self.items}
        
        num_users = len(train_data)
        
        for k in range(self.maxiter):
            s = "iter: " + str(k+1)
            print(s)
            #tqdm.write(s)
            #users = np.random.shuffle(self.users)
            i = 0
            for i in tq.tqdm(range(num_users)):
                u = np.random.choice(self.users)
                indices = np.arange(len(train_data[u]))
                rnd_idx = np.random.choice(indices)
                rnd_pair = train_data[u][rnd_idx]
                pos_item = rnd_pair[0]
                neg_item = rnd_pair[1]
                
                #retrieve factors
                u_factor = self.U[u]
                pos_factor = self.V[pos_item]
                neg_factor = self.V[neg_item]
                
                #make prediction
                x_uij = u_factor.dot(pos_factor.T - neg_factor.T)
                
                #compute gradient
                p = np.exp(-x_uij) / (1 + np.exp(-x_uij))
                grad_u = ((p * (pos_factor - neg_factor)) + self.lam * u_factor)
                grad_pos_v = ((p * u_factor) + self.lam * pos_factor)
                grad_neg_v = ((p * -u_factor) + self.lam * neg_factor)
                
                #update
                self.U[u] = u_factor + self.learning_rate * grad_u
                self.V[pos_item] = pos_factor + self.learning_rate * grad_pos_v
                self.V[neg_item] = neg_factor + self.learning_rate * grad_neg_v
                
            #compute validation on random slice:
            auc = 0
            for u in tq.tqdm(self.users):
                indices = np.arange(len(train_data[u]))
                rnd_idx = np.random.choice(indices)
                rnd_pair = train_data[u][rnd_idx]
                pos_item = rnd_pair[0]
                neg_item = rnd_pair[1]
                
                x_ui = self.U[u].dot(self.V[pos_item].T)
                x_uj = self.U[u].dot(self.V[neg_item].T)
                if x_ui > x_uj:
                    auc += 1
            auc = auc / num_users
            print("val AUC: {}".format(auc))
                

                
    def predict(self, user, test_item, neg_item):
        #implement
        u_factor = self.U[user]
        test_v_factor = self.V[test_item]
        neg_v_factor = self.V[neg_item]
        x_uij = u_factor.dot(test_v_factor.T) - u_factor.dot(neg_v_factor.T)
        pred = sigmoid(x_uij)
        return(pred)

In [218]:
bpr = BPR(learning_rate=0.05, maxiter=64, n_factors=124, lam=0.05)
bpr.fit(train_dict, item_list)

iter: 1



val AUC: 0.5085273617084384
iter: 2



val AUC: 0.503781699540264
iter: 3



val AUC: 0.5106035889070146
iter: 4



val AUC: 0.5138662316476346
iter: 5


...


val AUC: 0.7231202728755747
iter: 61



val AUC: 0.7330565030401899
iter: 62



val AUC: 0.7357259380097879
iter: 63



val AUC: 0.7335014088684562
iter: 64



val AUC: 0.7438825448613376


In [219]:
AUC = 0
user_sample = np.random.choice(list(train_dict.keys()), size=1000)
#user_sample = list(train_dict.keys())
user_auc_dict = {}
num_items_per_user = 8
for u in tq.tqdm(user_sample):
    user_auc = 0
    test_items = test_dict[u]
    for i in tq.tqdm(np.random.choice(test_items, size=num_items_per_user)):
        #rnd_item = np.random.choice(test_items)
        #rnd_item = i
        neg_test_items = list(set(item_list).difference(set(used_items_dict[u][0])
                                                    | set(used_items_dict[u][1])
                                                    | set(used_items_dict[u][2])))

        x_ui = bpr.U[u].dot(bpr.V[i].T)
        for j in neg_test_items:
            #prob = bpr.predict(u, rnd_item, neg_item)
            #user_auc += (prob > 0.50)*1
            x_uj = bpr.U[u].dot(bpr.V[j].T)
            pred = (x_ui > x_uj)*1
            user_auc += pred[0][0]
    #update user average auc      
    #user_auc = user_auc / (len(neg_test_items) * len(test_items))
    user_auc = user_auc / (len(neg_test_items) * num_items_per_user)
    user_auc_dict[u] = user_auc
    AUC += user_auc    #increment total auc
    
AUC = AUC / len(user_sample)

In [220]:
#print(AUC / len(user_sample))
print("Test AUC: {:.3f}".format(AUC))

Test AUC: 0.672


In [221]:
print(np.std(list(user_auc_dict.values())))

0.2383853113096599


In [222]:
len(list(user_auc_dict.keys()))

927

In [223]:
user_auc_dict

{2114: 0.34622587150677037,
 5874: 0.8932544209925841,
 2531: 0.33949471881244647,
 181: 0.4517199543248644,
 3849: 0.4503141062250143,
 4361: 0.6763131601484442,
 6223: 0.5293002289639381,
 1271: 0.2575085812356979,
 3106: 0.6271032782129388,
 4489: 0.8908041857798165,
 5299: 0.9630819765781206,
 3301: 0.8090168619605601,
 1879: 0.3438304260794967,
 1461: 0.6920610034207526,
 566: 0.2623108763916643,
 2438: 0.22380068532267275,
 6131: 0.7774764487582072,
 1835: 0.7089352788211499,
 424: 0.5645063597819503,
 1840: 0.6212697274031563,
 6457: 0.5485197368421053,
 4961: 0.7916547072330654,
 2247: 0.8561946902654868,
 5979: 0.999928632600628,
 4035: 0.5562731944047958,
 3105: 0.47202397944618896,
 5030: 0.6047463768115942,
 5735: 0.867036609974056,
 131: 0.5217581775700935,
 973: 0.6520482443619755,
 1874: 0.9270059965733867,
 5063: 0.768664485425342,
 6048: 0.753437410484102,
 180: 0.6837731811697575,
 587: 0.4904545454545455,
 1032: 0.26498715386811306,
 3802: 0.9685974734424346,
 3430: 

In [4]:
lam = 3082
from scipy.stats import poisson
poisson.pmf(5248, lam)

1.94454734015869e-275

In [10]:
N = 43
p = 1/2

((np.random.binomial(N, p, size=1000) - (N*p)) < 0).sum()

496